In [1]:
import os
import requests
from datetime import datetime
import pandas as pd

def fetch_fred_data(series_id, api_key,
                     observation_start="2000-01-01",
                     observation_end="2023-12-31",
                     realtime_start="2000-01-01",
                     realtime_end="2023-12-31"):
    """
    Récupère une série depuis l'API FRED en filtrant les observations entre observation_start et observation_end.
    Les paramètres realtime_start et realtime_end indiquent la date de vintage des données.
    """
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'observation_start': observation_start,
        'observation_end': observation_end,
        'realtime_start': realtime_start,
        'realtime_end': realtime_end
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  
    data = response.json()
    observations = data.get('observations', [])
    
    # Conversion des valeurs en numérique et des dates en datetime
    df = pd.DataFrame(observations)
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df['date'] = pd.to_datetime(df['date'])
    return df

# -------------------------
# Configuration et clé API
# -------------------------
fred_api_key = "f7c1d46211fba9fd51ba89b7e55d9f84"

# Définir la période souhaitée pour les observations
observation_start = "2000-01-01"
observation_end = "2023-12-31"

realtime_start = "2024-10-01"
realtime_end = "2024-10-01"

# -------------------------
# Récupérer les séries de données
# -------------------------
try:
    # 1. PIB des USA : GDPC1 (PIB en volume)
    df_gdp = fetch_fred_data("GDPC1", fred_api_key,
                             observation_start, observation_end,
                             realtime_start, realtime_end)
    df_gdp = df_gdp[['date', 'value']].rename(columns={'value': 'GDP'})
except requests.HTTPError as e:
    print("Erreur lors de la récupération de GDPC1. Vérifiez les dates et la clé API.")
    raise e

try:
    # 2. Emploi : PAYEMS (Effectifs non agricoles)
    df_emp = fetch_fred_data("PAYEMS", fred_api_key,
                             observation_start, observation_end,
                             realtime_start, realtime_end)
    df_emp = df_emp[['date', 'value']].rename(columns={'value': 'Employment'})
except requests.HTTPError as e:
    print("Erreur lors de la récupération de PAYEMS. Vérifiez les dates et la clé API.")
    raise e

try:
    # 3. Inflation (proxy) : CPIAUCSL (Indice des prix à la consommation)
    df_infl = fetch_fred_data("CPIAUCSL", fred_api_key,
                              observation_start, observation_end,
                              realtime_start, realtime_end)
    df_infl = df_infl[['date', 'value']].rename(columns={'value': 'CPI'})
except requests.HTTPError as e:
    print("Erreur lors de la récupération de CPIAUCSL. Vérifiez les dates et la clé API.")
    raise e

# -------------------------
# Fusionner les séries sur la date
# -------------------------
df_merged = pd.merge(df_gdp, df_emp, on='date', how='outer')
df_merged = pd.merge(df_merged, df_infl, on='date', how='outer')

# Trier par date
df_merged.sort_values(by='date', inplace=True)
df_merged.reset_index(drop=True, inplace=True)

# -------------------------
# Sauvegarder dans un fichier CSV
# -------------------------
today = datetime.today().strftime('%Y%m%d')
base_name, ext = os.path.splitext("usa_macro_data.csv")
output_file = os.path.join("macroeconomics", f"{base_name}_{today}{ext}")
df_merged.to_csv(output_file, index=False, encoding='utf-8')
print(f"Les données (GDP, Employment, CPI) ont été enregistrées dans '{output_file}'")

Les données (GDP, Employment, CPI) ont été enregistrées dans 'macroeconomics/usa_macro_data_20250320.csv'
